In [47]:
import character as char
import json
import pandas as pd

In [ ]:
john = char.PC(name= "John", 
char_class= "John", 
race= "John", 
level= "John", 
ability_scores= "John", 
max_hp= "John", 
current_hp= "John", 
armor_class= "John")

In [ ]:
# # Get unique official spells

# with open("../data/spells.json") as f:
#     data = json.load(f)

# df = pd.json_normalize(data)

# df_woc = df[df["publisher"]=="Wizards of the Coast"]
# df_woc = df_woc[~df_woc["book"].isin(["Player's Handbook", "Free Basic Rules (2024)", "Free Basic Rules (2014)", "Explorer's Guide to Wildemount (deprecated)",
# "(Deprecated) Rime of the Frostmaiden "])]
# woc_spells = df_woc.drop_duplicates(subset="name", keep="first").reset_index(drop=True)
# woc_spells = woc_spells.drop(columns=["publisher", "properties.Category"])
# woc_spells['properties.School'] = woc_spells['properties.School'].str.capitalize()
# woc_spells['properties.Damage Type'] = woc_spells['properties.Damage Type'].str.capitalize()
# woc_spells = woc_spells.dropna(axis=1, how='all')
# #woc_spells.to_csv("../data/woc_spells.csv")

In [81]:
df = pd.read_csv("../data/woc_spells.csv")

In [70]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         536 non-null    object
 1   description  536 non-null    object
 2   Level        536 non-null    int64 
 3   School       536 non-null    object
 4   Components   536 non-null    object
 5   DamageType   183 non-null    object
 6   CastingTime  536 non-null    object
 7   Range        291 non-null    object
 8   Save         190 non-null    object
dtypes: int64(1), object(8)
memory usage: 37.8+ KB


In [84]:
class Spell:
    def __init__(self, data):
        self.name = data["name"]
        self.description = data["description"]
        self.level = data["Level"]
        self.school = data["School"]
        self.components = data["Components"]
        self.dmg_type = data["DamageType"]
        self.cast_time = data["CastingTime"]
        self.range = data["Range"]
        self.save = data["Save"]


In [ ]:
SPELLS = {
    data["name"]: Spell(data)
    for spell_id, data in  df.to_dict(orient="index").items()
}

In [43]:
df = pd.read_csv("../data/woc_backgrounds.csv")

In [46]:
sum([1 for val in ["Fighter", "Fighter", "Fighter", "Rogue"] if val=="Fighter"])

3

In [39]:
df[df["name"]=="Aasimar"].iloc[0].to_dict()

{'name': 'Aasimar',
 'description': 'Whereas tieflings have fiendish blood in their veins, aasimar are the descendants of celestial beings. These folk generally appear as glorious humans with lustrous hair, flawless skin, and piercing eyes. Aasimar often attempt to pass as humans in order to right wrongs and defend goodness on the Material Plane without drawing undue attention to their celestial heritage. They strive to fit into society, although they usually rise to the top, becoming revered leaders and honorable heroes. You might decide to use the aasimar as a counterpoint to the tiefling race. The two races could even be at odds, reflecting some greater conflict between the forces of good and evil in your campaign. Here are our basic goals for the aasimar: Aasimar should make effective clerics and paladins. Aasimar should be to celestials and humans what tieflings are to fiends and humans. Given that aasimar and tieflings are like two sides of the same coin, the tiefling makes a goo

In [40]:
class Race:
    def __init__(self, id):
        df = pd.read_csv("../data/woc_races.csv")
        self.id = id
        if id not in df["name"].values:
            raise ValueError(f"{id} not a valid race.")
        self.racial_data=df[df["name"]==id].iloc[0].to_dict()


    def apply(self, character):
        character.ability_scores.apply_bonuses(self.ability_bonuses)

        for feature in self.features:
            character.features.add(feature)

In [24]:
import random as random
def roll_4d6_drop_lowest():
        result = []
        for _ in range(6):
            rolls = sorted([random.randint(1, 6) for _ in range(4)])
            result.append(sum(rolls[1:]))
         # Returns a list of six scores
        return result

roll_4d6_drop_lowest()

[17, 10, 17, 13, 12, 16]